In [1]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix

In [2]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0],dtype = np.complex128)
zeta = np.array([0,0],dtype = np.complex128)
theta = 0
psi = 0
theta1 = 0.3
psi1 = 0.1
cutoff_dim = 3
cutoff= 3
cutoff_plus = 10


In [3]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [4]:
old_state

array([[0.32663407+0.18432997j, 0.0201386 +0.1568399j ,
        0.36204319+0.11802914j],
       [0.03706083+0.08542767j, 0.44173217+0.29344608j,
        0.15211611+0.18573142j],
       [0.29873595+0.14419324j, 0.2462261 +0.35570142j,
        0.14166204+0.15660018j]])

In [5]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff+1, dtype=dtype))

G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

In [20]:
mu

array([ 0.+0.j,  0.+0.j, -0.-0.j, -0.-0.j])

In [6]:
Sigma

array([[ 0.        +0.j        ,  0.        +0.j        ,
        -0.95533649+0.j        ,  0.29404384-0.02950279j],
       [ 0.        +0.j        ,  0.        +0.j        ,
        -0.29404384-0.02950279j, -0.95533649+0.j        ],
       [-0.95533649+0.j        , -0.29404384-0.02950279j,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.29404384-0.02950279j, -0.95533649+0.j        ,
         0.        +0.j        ,  0.        +0.j        ]])

In [7]:
G_00pq

array([[1.+0.j, 0.-0.j, 0.+0.j],
       [0.-0.j, 0.+0.j, 0.-0.j],
       [0.+0.j, 0.-0.j, 0.+0.j]])

In [25]:
# R_00^jk = a_dagger^j \G_00pq> *  b^k|old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)

for k in range(cutoff):
    for j in range(cutoff):
        if j==0 and k ==0:
            R[0,0,j,k] = np.sum(G_00pq*old_state)

        elif j>0 and k==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,:] = G_00pq[:cutoff-j,:] * sqrt[1:cutoff-j+1,None]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

        elif k>0 and j==0:
            state = np.zeros((cutoff, cutoff), dtype=dtype)
            state[:cutoff-k,:] = old_state[k:,:] * sqrt[1:cutoff-k+1,None]
            R[0,0,j,k] = np.sum(G_00pq*state)

        else:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,:] = G_00pq[:cutoff-j,:] * sqrt[1:cutoff-j+1,None]
            state = np.zeros((cutoff, cutoff), dtype=dtype)
            state[:cutoff-k,:] = old_state[k:,:] * sqrt[1:cutoff-k+1,None]
            R[0,0,j,k] = np.sum(G_00pq2*state)
            
print("oldstate",old_state)

print(R[0,0,:cutoff,:cutoff])    
#R_0n^jk
for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(1,cutoff):
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]
print("=====")
print(R[0,1,:cutoff,:cutoff])   

for k in range(0,cutoff):
    for j in range(0,cutoff):
        for n in range(0,cutoff):
            for m in range(1,cutoff):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]


oldstate [[0.32663407+0.18432997j 0.0201386 +0.1568399j  0.36204319+0.11802914j]
 [0.03706083+0.08542767j 0.44173217+0.29344608j 0.15211611+0.18573142j]
 [0.29873595+0.14419324j 0.2462261 +0.35570142j 0.14166204+0.15660018j]]
[[0.32663407+0.18432997j 0.03706083+0.08542767j 0.29873595+0.14419324j]
 [0.03706083+0.08542767j 0.42247644+0.20392003j 0.        +0.j        ]
 [0.29873595+0.14419324j 0.        +0.j         0.        +0.j        ]]
=====
[[0.04378272+0.10782505j 0.40360374+0.21017872j 0.        +0.j        ]
 [0.48719452+0.24602492j 0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j        ]]


In [26]:
R[:,:,:cutoff,:cutoff]

array([[[[0.32663407+0.18432997j, 0.03706083+0.08542767j,
          0.29873595+0.14419324j],
         [0.03706083+0.08542767j, 0.42247644+0.20392003j,
          0.        +0.j        ],
         [0.29873595+0.14419324j, 0.        +0.j        ,
          0.        +0.j        ]],

        [[0.04378272+0.10782505j, 0.40360374+0.21017872j,
          0.        +0.j        ],
         [0.48719452+0.24602492j, 0.        +0.j        ,
          0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ,
          0.        +0.j        ]],

        [[0.        +0.j        , 0.        +0.j        ,
          0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ,
          0.        +0.j        ],
         [0.        +0.j        , 0.        +0.j        ,
          0.        +0.j        ]]],


       [[[0.0219877 +0.05758609j, 0.31151159+0.16122666j,
          0.        +0.j        ],
         [0.15515055+0.09025587j, 0.        +0.j        ,
    

In [11]:
R[:,:,0,0]

array([[0.32663407+0.18432997j, 0.04378272+0.10782505j,
        0.        +0.j        ],
       [0.0219877 +0.05758609j, 0.34055665+0.18514226j,
        0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        ]])

In [12]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [13]:
np.einsum("cd,abcd->ab",old_state,G)

array([[0.32663407+0.18432997j, 0.02761629+0.17604776j,
        0.51728191+0.25943026j],
       [0.02485672+0.03608851j, 0.32897524+0.27892414j,
        0.2148429 +0.32535478j],
       [0.11849961+0.0364519j , 0.08633727+0.16366631j,
        0.07391477+0.08170901j]])

In [14]:
np.sum(G[0,0,:,:]*old_state)

(0.32663407103801245+0.18432997262524056j)

In [15]:
np.sum(G[0,1,:,:]*old_state)

(0.027616292224567407+0.17604775727930336j)

In [16]:
np.sum(G[1,1,:,:]*old_state)

(0.3289752431890487+0.2789241390804018j)